# Sentiment Analysis Inference APP with GRADIO
Sentiment Analysis is the process of ‘computationally’ determining whether a piece of writing is positive, negative or neutral. It’s also known as opinion mining, deriving the opinion or attitude of a speaker.

<img src= "https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/sentiment.jpg" width=1000>

In [1]:
!pip install onnxruntime -q
!pip install gradio==4.15.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s e

In [2]:
!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/tokenizer_sentiment.pkl
!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/sentiment-int8.onnx

--2024-02-28 18:27:18--  https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/tokenizer_sentiment.pkl
Resolving frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)... 3.5.245.154, 52.95.191.6, 52.95.191.62, ...
Connecting to frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)|3.5.245.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 468214 (457K) [application/octet-stream]
Saving to: ‘tokenizer_sentiment.pkl’

tokenizer_sentiment 100%[===================>] 457.24K  --.-KB/s    in 0.05s   

2024-02-28 18:27:19 (8.72 MB/s) - ‘tokenizer_sentiment.pkl’ saved [468214/468214]

--2024-02-28 18:27:19--  https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/sentiment-int8.onnx
Resolving frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)... 3.5.245.154, 52.95.191.6, 52.95.191.62, ...
Connecting to frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)|3.5.245.154|:443... connected

In [3]:
### MAIN SCRIPT OF INFERENCE  #######
# ONLY 3 LIB. REQUIRES

import onnxruntime
import joblib
import numpy as np

tokenizer = joblib.load("tokenizer_sentiment.pkl")  # load tokenizer
onnx_model_path = "sentiment-int8.onnx"            # load model quantized int8
ort_session = onnxruntime.InferenceSession(onnx_model_path)

def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum(axis=-1, keepdims=True)

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt")

    # Prepare input data for ONNX model
    input_data = {
                "input_ids": inputs["input_ids"].numpy(),
                "attention_mask": inputs["attention_mask"].numpy()
                }
    output = ort_session.run(None, input_data)
    logits = np.array(output[0])
    probabilities = softmax(logits)
    sentiment_label = np.argmax(probabilities).item()
    sentiment_prob = np.max(probabilities).item()
    # Map sentiment labels to "Negative" and "Positive"
    if sentiment_label == 0:
        sentiment_str = "Negative"
    else:
        sentiment_str = "Positive"
    return sentiment_str, sentiment_prob

In [10]:
## UI OF INFERENCE INSIDE COLAB

import gradio as gr

title = "Sentiment Analysis Application"
description = "This application analyzes the sentiment of the given text with DistillBert Quantized (8-bit)."

demo = gr.Interface(
                    fn=analyze_sentiment,
                    inputs=gr.Textbox(placeholder="Write your text here...", label="Input Text"),
                    outputs=[gr.Textbox(label="Sentiment"), gr.Textbox(label="Sentiment Probability")],
                    title=title,
                    description=description,
                    allow_flagging= "never"
                    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1bb787aa1dae6d507b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
